In [ ]:
import torch
import timm

# 1. 모델 생성 (학습에 사용한 것과 동일해야 함)
model = timm.create_model("deit_base_patch16_224", pretrained=False, num_classes=4, drop_rate=0.3)
model.load_state_dict(torch.load("/workspace/models/DeiT/Deit_finetuned_drawing.pth"))
model.eval().cuda()  # GPU 사용 시

# 2. 더미 입력 텐서 정의 (224x224 RGB 이미지 한 장)
dummy_input = torch.randn(1, 3, 224, 224).cuda()

# 3. ONNX로 저장
torch.onnx.export(
    model,
    dummy_input,
    "deit_drawing.onnx",  # 저장 파일명
    input_names=["input"],
    output_names=["output"],
    opset_version=15,
    export_params=True,
    do_constant_folding=False # 상수 폴더(옵티마이저랑 비슷?)
)

print("✅ ONNX 변환 완료: deit_drawing2.onnx")


✅ ONNX 변환 완료: deit_drawing2.onnx


In [8]:
class_info = {
    "0": "angry",
    "1": "fear",
    "2": "happy",
    "3": "sad"
}

# 저장
with open("class_info.json", "w") as f:
    json.dump(class_info, f)

In [11]:
import onnxruntime as ort
from PIL import Image
import torchvision.transforms as transforms
import torch
import numpy as np
import json

# ✅ 1. 모델 로드
onnx_model_path = "deit_drawing.onnx"
session = ort.InferenceSession(onnx_model_path)

# ✅ 2. 클래스 정보 로드
with open("class_info.json", "r") as f:
    class_info = json.load(f)  # { "0": "happy", "1": "sad", ... }

# ✅ 3. 이미지 전처리 함수 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # DeiT는 보통 mean=0.5, std=0.5 사용
])

# ✅ 4. 이미지 불러오기 및 전처리
img = Image.open("/workspace/2.png").convert("RGB")
input_tensor = transform(img).unsqueeze(0).numpy()  # (1, 3, 224, 224)

# ✅ 5. ONNX 모델에 입력
inputs = {session.get_inputs()[0].name: input_tensor}
outputs = session.run(None, inputs)

# ✅ 6. 예측 결과 해석
logits = outputs[0]
pred_class = int(np.argmax(logits))
pred_label = class_info[str(pred_class)]

print(f"✅ 예측 결과: {pred_label}")


✅ 예측 결과: sad
